In [4]:
import mimetypes

mime_type, _ = mimetypes.guess_type("test add.pdf")

mime_type

'application/pdf'

In [4]:
from enum import StrEnum

class SubmissionStatus(StrEnum):
    PROCESSING = "PROCESSING"  # the request is being processed to be extracted
    INREVIEW = "IN_REVIEW"  # the request is in review
    CLOSED = "CLOSED"  # the request got rejected
    ERROR = "ERROR"  # the request extraction processing failed
    DUPLICATE = "DUPLICATE"  # already have it

class ClaimStatus(StrEnum):
    APPROVED = "APPROVED"  # we want this business
    DECLINED = "DECLINED"  # we don’t want it
    PENDING = "PENDING"  # still deciding
    PARTIAL_PAYMENT = "PARTIAL_PAYMENT"
    DECIDING = "DECIDING"
    MISSING = "MISSING"  # missing information
    CLOSED = "CLOSED"
    PAID = "PAID"


In [54]:
from typing import Any, Dict
from dateutil import parser


def _nullify_structure(value: Any) -> Any:
    """Recursively replace leaf values with None, preserving dict/list shape."""
    if isinstance(value, dict):
        return {k: _nullify_structure(v) for k, v in value.items()}
    if isinstance(value, list):
        return [_nullify_structure(v) for v in value]
    # primitives (str, int, float, bool, None, etc.)
    return None


def override_sections_with_none(
    data: dict[str, Any], section_key: str
) -> dict[str, Any]:
    """For each top-level `<name>_data` in `data`, if corresponding `<name>_doc` is falsy (None/empty),
    replace that section with a shape-preserving version where all leaves are None.
    """

    section_value = data.get(section_key)
    data[section_key] = _nullify_structure(section_value)
    return data


async def _process_request_info(
    extracted_data: dict, missing_documents: list[str]
) -> dict:

    policy_data = extracted_data["policy_data"]
    claim_data = extracted_data["claim_data"]
    decision_data = extracted_data["decision_data"]

    client_name = claim_data["claim_details"]["first_name"]
    last_name = claim_data["claim_details"]["last_name"]
    phone_number = claim_data["claim_details"]["phone_number"]
    mobile_number = claim_data["claim_details"]["mobile_number"]


    client_name = (
        f"{client_name} {last_name}".strip() if client_name or last_name else None
    )

    if phone_number is not None:
        phone_number = mobile_number

    request_data = {
        "missing_documents": missing_documents,
        # Claim Info
        "claim_number": claim_data["claim_details"]["claim_number"],
        "type_of_claim": claim_data["claim_reason"]["claim_type"],
        "requested_reimbursement_amount": str(
            claim_data["claimants_expense"]["total_expected_refunds"]
        ),
        "claim_amount": str(claim_data["claimants_expense"]["total_claimed_amount"]),

        # Contact Info
        "client_name": client_name,
        "business_number": claim_data["claim_details"]["business_number"],
        "client_phone_number": phone_number,
        "client_post_code": claim_data["claim_details"]["zip_postal_code"],
        # Decision Info
    }

    if decision_data:
        fraud_reasons = decision_data["fraud_and_amount_check"]["fraud_reasons"]
        premium_amount = decision_data["premium_amount_check"]["premium_amount"]
        if premium_amount and not isinstance(premium_amount, str):
            premium_amount = str(premium_amount)
        if fraud_reasons and isinstance(fraud_reasons, list):
            fraud_reasons = ". \n".join(str(reason) for reason in fraud_reasons if reason)
        elif fraud_reasons is None:
            fraud_reasons = ""
        decision_info = {
            "generated_decision": decision_data["decision_reason"],
            "decision_confidence_level":decision_data["confidence_level"],
            "is_fraud_suspected": decision_data["fraud_and_amount_check"]["is_fraud_suspected"],
            "payment_reason": decision_data["payment_check"]["payment_reason"],
            "fraud_reasons": fraud_reasons,
            "summary_of_findings": decision_data["summary_of_findings"],
            "case_description": decision_data["case_description"],
            "approved_amount": f"{decision_data['fraud_and_amount_check']['approved_amount']}",
            "premium_amount":premium_amount ,
        }

        # Fraud detection
        matched_coverage_label = decision_data["refunds_check"].get(
            "matched_coverage_label"
        )
        matched_coverage_terms = decision_data["refunds_check"].get(
            "matched_coverage_terms"
        )

        if matched_coverage_label and matched_coverage_terms:
            matched_coverage_label = matched_coverage_label.capitalize().replace("_", " ")
            request_data["matched_coverage_terms"] = (
                f"{matched_coverage_label}: {matched_coverage_terms}"
            )
        request_data = {**request_data, **decision_info}

    if policy_data:
        policy_info = {
            # Policy Info
            "policy_number": policy_data["policy_info"]["policy_number"],
            "policy_holder_name": policy_data["contact_info"]["policy_holder_name"],
            "policy_total_cost": policy_data["policy_info"]["policy_total_cost"],
            # trip info
            "trip_cost": policy_data["trip_details"]["trip_cost"],
            "destination": policy_data["trip_details"]["trip_destination"],
            "client_email_address": policy_data["contact_info"]["email"],
        }
        request_data = {**request_data, **policy_info}

        departure_date_str = policy_data["trip_details"]["departure_date"]
        return_date_str = policy_data["trip_details"]["return_date"]
        policy_effective_date_str = policy_data["policy_info"]["coverage_effective_date"]
        policy_expiration_date_str = policy_data["policy_info"]["coverage_expiration_date"]
        if departure_date_str:
            departure_date = parser.parse(departure_date_str).replace(tzinfo=None)
            request_data["trip_start_date"] = departure_date

        if return_date_str:
            return_date = parser.parse(return_date_str).replace(tzinfo=None)
            request_data["trip_return_date"] = return_date

        if policy_effective_date_str:
            policy_effective_date = parser.parse(policy_effective_date_str).replace(
                tzinfo=None
            )
            request_data["policy_effective_date"] = policy_effective_date

        if policy_expiration_date_str:
            policy_expiration_date = parser.parse(policy_expiration_date_str).replace(
                tzinfo=None
            )
            request_data["policy_expiration_date"] = policy_expiration_date


    is_duplicated = False

    # REQUEST APPETITE STATUS
    if is_duplicated:
        request_data["submission_status"] = SubmissionStatus.DUPLICATE
    else:
        request_data["submission_status"] = SubmissionStatus.INREVIEW

    if len(missing_documents) > 0:
        request_data["missing_documents"] = missing_documents
        request_data["status"] = ClaimStatus.MISSING
    elif decision_data is not None:
        appetite_status = decision_data["appetite"]
        if appetite_status == "approve":
            payment_status = decision_data["payment_check"]["payment_status"]
            if payment_status == "partial_payment":
                request_data["status"] = ClaimStatus.PARTIAL_PAYMENT
            elif payment_status == "full_payment":
                request_data["status"] = ClaimStatus.APPROVED
            else:
                request_data["status"] = ClaimStatus.APPROVED
        else:
            request_data["status"] = ClaimStatus.DECLINED
    return request_data


data = {
    "hotel_data": {
        "room_type": "Room (Studio)",
        "booking_details": "2 nights • 2 adults • 1 room",
        "nights_count": 2,
        "checkin_checkout": [
            {"checkin_date": "2025-09-16", "checkout_date": "2025-09-18"}
        ],
        "total": 296.27,
    },
    "policy_data": {
        "policy_info": {
            "policy_number": "[HEALTHCARE_NUMBER_1]",
            "policy_name": "Hotel Booking Protection Plan",
            "coverage_effective_date": "2025-09-10",
            "coverage_expiration_date": "2025-09-11",
            "policy_total_cost": "$12.89",
        },
        "trip_details": {
            "departure_date": "2025-09-16",
            "return_date": "2025-09-18",
            "trip_cost": "$283.38",
            "trip_destination": None,
        },
        "contact_info": {
            "policy_holder_name": "[NAME_1]",
            "email": "[EMAIL_ADDRESS_1]",
        },
        "coverages_and_benefit_limits": {
            "standard_packages": {
                "trip_cancellation": "100% Per Booking",
                "trip_interruption": "100% Per Booking",
                "trip_delay": "$150 Per Insured",
                "ancillary_evacuation_benefits": "$2,500 Per Insured",
                "baggage_delay": "$150 Per Insured",
                "travel_medical_assistance": "Included Per Insured",
                "worldwide_travel_assistance": "Included Per Insured",
            }
        },
    },
    "claim_data": {
        "claim_details": {
            "claim_number": "UC8116302172",
            "title": "M.",
            "first_name": "[NAME_GIVEN_1]",
            "last_name": "[NAME_FAMILY_1]",
            "date_of_birth": "",
            "street_address_1": "[LOCATION_ADDRESS_STREET_2]",
            "street_address_2": "[LOCATION_ADDRESS_STREET_2]",
            "city": "Any",
            "state_province": "co",
            "zip_postal_code": "[LOCATION_ZIP_2]",
            "country_code": "US",
            "country": "UNITED STATES",
            "email_address": "[EMAIL_ADDRESS_1]",
            "phone_number": "[PHONE_NUMBER_3]",
            "mobile_number": "",
            "business_number": "",
            "external_reference": "",
        },
        "travel_details": {
            "scheduled_departure_date": "2025-09-16",
            "scheduled_return_date": "2025-09-18",
            "booking_date": "2025-09-16",
            "incident_date": "2025-09-10",
            "trip_country": "UNITED STATES",
        },
        "claim_reason": {
            "claim_type": "trip_interruption",
            "claim_reason_type": "illness",
            "type_of_loss": "----Illness---- Whose plans were interrupted or delayed? * [NAME_1] -- Who became ill? * [NAME_1] What date did the ill person first begin to have symptoms of illness? * 09/08/2025 When did the interruption occur?* 09/10/2025 Please describe the illness and circumstance that led to the interruption or delay. * Covid 19 test ----Expenses---- Do you have any other insurance coverage for this loss?* No Do you have any medical expenses related to your interruption or delay?* No",
        },
        "claimants_expense": {
            "claim_amount": 296.27,
            "expense_type": "Unused Hotel/Resort",
            "claimed_interruption_expenses": 296.27,
            "claimed_medical_expenses": 0,
            "total_claimed_amount": 296.27,
            "total_expected_refunds": 150,
            "net_total_claimed_amount": 146.26999999999998,
        },
    },
}


decision_data = {
    "policy_limits": {
        "per_day_limit": None,
        "overall_cap_per_person": None,
        "overall_cap_per_trip": 100000,
        "minimum_delay_hours": 5,
    },
    "eligibility_window": {"eligible_days": 2, "eligible_persons": 1},
    "fraud_and_amount_check": {
        "is_fraud_suspected": True,
        "fraud_reasons": ["missing medical proof for illness"],
        "claim_total": 296.27,
        "policy_claimable_max": 150.0,
        "within_limit": False,
        "delta": 146.27,
        "capped_payout": 150.0,
        "approved_amount": 0.0,
    },
    "refunds_check": {
        "expected_refund": 150.0,
        "protection_plan_cost": 12.89,
        "refund_net_of_premium": 137.11,
        "matched_coverage_label": "trip_interruption",
        "matched_coverage_terms": "100% Per Booking",
    },
    "payment_check": {
        "payment_status": "not_resolved",
        "payment_reason": "Claim declined due to lack of required medical documentation for an illness-based claim.",
    },
    "premium_amount_check": {"premium_amount": 12.89},
    "appetite": "decline",
    "decision_reason": "Missing medical evidence for illness-based cancellation",
    "confidence_level": 0.7,
    "summary_of_findings": "The claim for trip interruption due to illness is declined. While the policy provides coverage for such events, the claimant failed to submit the required medical evidence (e.g., a positive Covid-19 test result) to verify the Sickness. As per policy requirements and fraud prevention protocols, the claim cannot be approved without this documentation. The claimed amount of $296.27 also exceeds the expected refund cap of $150.",
    "case_description": "The claimant filed for trip interruption due to testing positive for Covid-19 before their scheduled trip. The claim is for the cost of an unused hotel stay. The policy covers trip interruption for Sickness, but requires verification. The claimant has not provided any medical documentation, such as a positive test result, to substantiate the claim.",
}
extracted_data = data
extracted_data["decision_data"] = decision_data
extracted_data = override_sections_with_none(data, "decision_data")




In [55]:
missing_documents = ["proof of illness"]

res = await _process_request_info(extracted_data, missing_documents)
res

{'missing_documents': ['proof of illness'],
 'claim_number': 'UC8116302172',
 'type_of_claim': 'trip_interruption',
 'requested_reimbursement_amount': '150',
 'claim_amount': '296.27',
 'client_name': '[NAME_GIVEN_1] [NAME_FAMILY_1]',
 'business_number': '',
 'client_phone_number': '',
 'client_post_code': '[LOCATION_ZIP_2]',
 'generated_decision': None,
 'decision_confidence_level': None,
 'is_fraud_suspected': None,
 'payment_reason': None,
 'fraud_reasons': '',
 'summary_of_findings': None,
 'case_description': None,
 'approved_amount': 'None',
 'premium_amount': None,
 'policy_number': '[HEALTHCARE_NUMBER_1]',
 'policy_holder_name': '[NAME_1]',
 'policy_total_cost': '$12.89',
 'trip_cost': '$283.38',
 'destination': None,
 'client_email_address': '[EMAIL_ADDRESS_1]',
 'trip_start_date': datetime.datetime(2025, 9, 16, 0, 0),
 'trip_return_date': datetime.datetime(2025, 9, 18, 0, 0),
 'policy_effective_date': datetime.datetime(2025, 9, 10, 0, 0),
 'policy_expiration_date': datetime.d

In [56]:
travel_guard_required_files = {
    "trip_cancellation": {
        "illness": [
            "booking_confirmation",
            "supporting_claim_evidence_documentation",
            "proof_of_cancellation",
            "proof_of_payment_for_claimed_expenses",
            "documentation_of_diagnosis_and_treatment_dates",
        ],
        "injury": [
            "booking_confirmation",
            "supporting_claim_evidence_documentation",
            "proof_of_cancellation",
            "proof_of_payment_for_claimed_expenses",
            "documentation_of_diagnosis_and_treatment_dates",
        ],
        "pregnancy": [
            "booking_confirmation",
            "supporting_claim_evidence_documentation",
            "proof_of_cancellation",
            "proof_of_payment_for_claimed_expenses",
            "documentation_of_diagnosis_and_treatment_dates",
        ],
        "death": [
            "booking_confirmation",
            "supporting_claim_evidence_documentation",
            "proof_of_cancellation",
            "proof_of_payment_for_claimed_expenses",
            "death_certificate",
        ],
        "work": [
            "booking_confirmation",
            "supporting_claim_evidence_documentation",
            "proof_of_cancellation",
            "proof_of_payment_for_claimed_expenses",
            "evidence_of_work_employment_reasons",
        ],
        "military": [
            "booking_confirmation",
            "supporting_claim_evidence_documentation",
            "proof_of_cancellation",
            "proof_of_payment_for_claimed_expenses",
            "official_military_order_or_letter",
        ],
        "uninhabitable": [
            "booking_confirmation",
            "supporting_claim_evidence_documentation",
            "proof_of_cancellation",
            "proof_of_payment_for_claimed_expenses",
            "police_report",
            "evidence_of_damage_to_home_or_destination",
            "proof_of_residential_address",
        ],
        "court": [
            "booking_confirmation",
            "supporting_claim_evidence_documentation",
            "proof_of_cancellation",
            "proof_of_payment_for_claimed_expenses",
            "court_order_jury_summons_or_subpoena",
        ],
        "weather": [
            "booking_confirmation",
            "supporting_claim_evidence_documentation",
            "proof_of_cancellation",
            "proof_of_payment_for_claimed_expenses",
            "travel_carrier_statement",
        ],
        "carrier": [
            "booking_confirmation",
            "supporting_claim_evidence_documentation",
            "proof_of_cancellation",
            "proof_of_payment_for_claimed_expenses",
        ],
        "other": [
            "booking_confirmation",
            "supporting_claim_evidence_documentation",
            "proof_of_cancellation",
            "proof_of_payment_for_claimed_expenses",
        ],
    },
    "trip_interruption": {
        "illness": [
            "booking_confirmation",
            "supporting_claim_evidence_documentation",
            "confirmation_of_interruption",
            "proof_of_payment_for_claimed_expenses",
            "documentation_of_diagnosis_and_treatment_dates",
        ],
        "injury": [
            "booking_confirmation",
            "supporting_claim_evidence_documentation",
            "confirmation_of_interruption",
            "proof_of_payment_for_claimed_expenses",
            "documentation_of_diagnosis_and_treatment_dates",
        ],
        "pregnancy": [
            "booking_confirmation",
            "supporting_claim_evidence_documentation",
            "confirmation_of_interruption",
            "proof_of_payment_for_claimed_expenses",
            "documentation_of_diagnosis_and_treatment_dates",
        ],
        "death": [
            "booking_confirmation",
            "supporting_claim_evidence_documentation",
            "confirmation_of_interruption",
            "proof_of_payment_for_claimed_expenses",
            "death_certificate",
        ],
        "work": [
            "booking_confirmation",
            "supporting_claim_evidence_documentation",
            "confirmation_of_interruption",
            "proof_of_payment_for_claimed_expenses",
            "evidence_of_work_employment_reasons",
        ],
        "military": [
            "booking_confirmation",
            "supporting_claim_evidence_documentation",
            "confirmation_of_interruption",
            "proof_of_payment_for_claimed_expenses",
            "official_military_order_or_letter",
        ],
        "uninhabitable": [
            "booking_confirmation",
            "supporting_claim_evidence_documentation",
            "confirmation_of_interruption",
            "proof_of_payment_for_claimed_expenses",
            "police_report",
            "evidence_of_damage_to_home_or_destination",
            "proof_of_residential_address",
        ],
        "court": [
            "booking_confirmation",
            "supporting_claim_evidence_documentation",
            "confirmation_of_interruption",
            "proof_of_payment_for_claimed_expenses",
            "court_order_jury_summons_or_subpoena",
        ],
        "weather": [
            "booking_confirmation",
            "supporting_claim_evidence_documentation",
            "confirmation_of_interruption",
            "proof_of_payment_for_claimed_expenses",
            "travel_carrier_statement",
        ],
        "carrier": [
            "booking_confirmation",
            "supporting_claim_evidence_documentation",
            "confirmation_of_interruption",
            "proof_of_payment_for_claimed_expenses",
        ],
        "other": [
            "booking_confirmation",
            "supporting_claim_evidence_documentation",
            "confirmation_of_interruption",
            "proof_of_payment_for_claimed_expenses",
        ],
    },
    "trip_delay": {
        "illness": [
            "booking_confirmation",
            "supporting_claim_evidence_documentation",
            "confirmation_of_interruption",
            "proof_of_payment_for_claimed_expenses",
            "documentation_of_diagnosis_and_treatment_dates",
        ],
        "injury": [
            "booking_confirmation",
            "supporting_claim_evidence_documentation",
            "confirmation_of_interruption",
            "proof_of_payment_for_claimed_expenses",
            "documentation_of_diagnosis_and_treatment_dates",
        ],
        "pregnancy": [
            "booking_confirmation",
            "supporting_claim_evidence_documentation",
            "confirmation_of_interruption",
            "proof_of_payment_for_claimed_expenses",
            "documentation_of_diagnosis_and_treatment_dates",
        ],
        "death": [
            "booking_confirmation",
            "supporting_claim_evidence_documentation",
            "confirmation_of_interruption",
            "proof_of_payment_for_claimed_expenses",
            "death_certificate",
        ],
        "work": [
            "booking_confirmation",
            "supporting_claim_evidence_documentation",
            "confirmation_of_interruption",
            "proof_of_payment_for_claimed_expenses",
            "evidence_of_work_employment_reasons",
        ],
        "military": [
            "booking_confirmation",
            "supporting_claim_evidence_documentation",
            "confirmation_of_interruption",
            "proof_of_payment_for_claimed_expenses",
            "official_military_order_or_letter",
        ],
        "uninhabitable": [
            "booking_confirmation",
            "supporting_claim_evidence_documentation",
            "confirmation_of_interruption",
            "proof_of_payment_for_claimed_expenses",
            "police_report",
            "evidence_of_damage_to_home_or_destination",
            "proof_of_residential_address",
        ],
        "court": [
            "booking_confirmation",
            "supporting_claim_evidence_documentation",
            "confirmation_of_interruption",
            "proof_of_payment_for_claimed_expenses",
            "court_order_jury_summons_or_subpoena",
        ],
        "weather": [
            "booking_confirmation",
            "supporting_claim_evidence_documentation",
            "confirmation_of_interruption",
            "proof_of_payment_for_claimed_expenses",
            "travel_carrier_statement",
        ],
        "carrier": [
            "booking_confirmation",
            "supporting_claim_evidence_documentation",
            "confirmation_of_interruption",
            "proof_of_payment_for_claimed_expenses",
        ],
        "other": [
            "booking_confirmation",
            "supporting_claim_evidence_documentation",
            "confirmation_of_interruption",
            "proof_of_payment_for_claimed_expenses",
        ],
    },
    # OUT OF SCOPE
    # "baggage": {
    #     "loss": [
    #         "booking_confirmation",
    #         "supporting_claim_evidence_documentation",
    #         "report_from_liable_party_or_police",
    #         "evidence_of_damages",
    #         "repair_quotation_for_damaged_items",
    #         "purchase_receipt_of_damaged_item",
    #         "proof_of_value_and_ownership_of_damaged_property",
    #     ],
    #     "delay": [
    #         "booking_confirmation",
    #         "supporting_claim_evidence_documentation",
    #         "report_from_liable_party_or_police",
    #         "evidence_of_damages",
    #         "repair_quotation_for_damaged_items",
    #         "purchase_receipt_of_damaged_item",
    #         "proof_of_value_and_ownership_of_damaged_property",
    #         "receipts_for_essential_purchases",
    #         "property_irregularity_report",
    #         "confirmation_of_returned_luggage",
    #         "carrier_reimbursement_documentation",
    #     ],
    # },
    # "medical": {
    #     "medical": [
    #         "booking_confirmation",
    #         "supporting_claim_evidence_documentation",
    #         "documentation_of_diagnosis_and_treatment_dates",
    #         "medical_expense_receipts",
    #         "death_certificate_if_applicable",
    #     ]
    # },
    # "cdw": {
    #     "cdw": [
    #         "booking_confirmation",
    #         "supporting_claim_evidence_documentation",
    #         "proof_of_comprehensive_motor_insurance",
    #         "policy_wording_of_comprehensive_motor_insurance",
    #         "proof_of_payment_to_rental_agency_for_damages",
    #         "photos_of_damaged_vehicle",
    #         "police_report_and_rental_agency_incident_report",
    #         "driving_license",
    #     ]
    # },
}


In [57]:
 
claim_type = "trip_interruption"
claim_status ="illness"

check_file_list = travel_guard_required_files[claim_type][claim_status]
check_file_list


['booking_confirmation',
 'supporting_claim_evidence_documentation',
 'confirmation_of_interruption',
 'proof_of_payment_for_claimed_expenses',
 'documentation_of_diagnosis_and_treatment_dates']

In [59]:
test = {
   "final_missing_status": True,
   "missing_status": {
     "booking_confirmation": {
       "missing": False,
       "reason": "The Expedia screenshots serve as a booking confirmation, showing the hotel, dates, and itinerary number."
     },
     "supporting_claim_evidence_documentation": {
       "missing": True,
       "reason": "No documentation from the employer was provided to confirm the claimant was required to work."
     },
     "proof_of_cancellation": {
       "missing": False,
       "reason": "The Expedia booking screenshot clearly shows a 'Canceled' status for the reservation."
     },
     "proof_of_payment_for_claimed_expenses": {
       "missing": True,
       "reason": "No receipt or bank statement was provided to show the claimed hotel expense of $143.97 was paid."
     },
     "documentation_of_diagnosis_and_treatment_dates": {
       "missing": True,
       "reason": "No medical documentation was provided for the colleague whose sickness is cited as the reason for the work conflict."
     }
   },
   "generated_decision_summary": "The claim is missing key evidence, including proof of payment, employer verification of the work requirement, and medical documentation for the sick colleague, which are all needed to validate the claim."
 }


In [4]:
from pydantic import BaseModel

class MissingStatus(BaseModel):
    missing: bool
    reason: str

 

class MissingCheckOutcome(BaseModel):

    final_missing_status: bool
    missing_status : dict[str, MissingStatus]
    generated_decision_summary: str | None = None


In [5]:
data = {
   "final_missing_status": False,
   "missing_status": {
     "booking_confirmation": {
       "missing": False,
       "reason": "The AARP Travel Center receipt confirms the hotel booking details, including dates, location, and traveler name."
     },
     "supporting_claim_evidence_documentation": {
       "missing": False,
       "reason": "An obituary was provided as evidence for the trip cancellation reason, which was a death in the family."
     },
     "proof_of_cancellation": {
       "missing": False,
       "reason": "A cancellation confirmation email from AARP Travel Center for the hotel booking is present."
     },
     "proof_of_payment_for_claimed_expenses": {
       "missing": False,
       "reason": "The AARP Travel Center receipt shows the total amount was paid via American Express."
     }
   }
 }

MissingCheckOutcome(**data)

MissingCheckOutcome(final_missing_status=False, missing_status={'booking_confirmation': MissingStatus(missing=False, reason='The AARP Travel Center receipt confirms the hotel booking details, including dates, location, and traveler name.'), 'supporting_claim_evidence_documentation': MissingStatus(missing=False, reason='An obituary was provided as evidence for the trip cancellation reason, which was a death in the family.'), 'proof_of_cancellation': MissingStatus(missing=False, reason='A cancellation confirmation email from AARP Travel Center for the hotel booking is present.'), 'proof_of_payment_for_claimed_expenses': MissingStatus(missing=False, reason='The AARP Travel Center receipt shows the total amount was paid via American Express.')}, generated_decision_summary=None)

In [ ]:
import tempfile


content = "SOME TEXT"



with tempfile.NamedTemporaryFile(mode="w", suffix=".txt") as f:
    f.write(content)
    f.flush()
    print(f.name)




/tmp/tmpnlvmqnnk.txt


In [ ]:
data = {
  "policy_limits": {
    "per_day_limit": None,
    "overall_cap_per_person": None,
    "overall_cap_per_trip": 100000.0,
    "minimum_delay_hours": None
  },
  "eligibility_window": {
    "eligible_days": None,
    "eligible_persons": 1
  },
  "fraud_and_amount_check": {
    "is_fraud_suspected": True,
    "fraud_reasons": [
      "missing medical proof for illness"
    ],
    "claim_total": 664.12,
    "policy_claimable_max": 664.12,
    "within_limit": True,
    "delta": 0.0,
    "capped_payout": None,
    "approved_amount": 0.0
  },
  "refunds_check": {
    "expected_refund": 0.0,
    "protection_plan_cost": 12.89,
    "refund_net_of_premium": -12.89,
    "matched_coverage_label": "trip_cancellation",
    "matched_coverage_terms": "up to 100% of insured Trip Cost"
  },
  "payment_check": {
    "payment_status": "not_resolved",
    "payment_reason": "Claim declined due to missing medical evidence required by the policy for an illness-based cancellation."
  },
  "premium_amount_check": {
    "premium_amount": 12.89
  },
  "appetite": "decline",
  "decision_reason": "Missing medical evidence in attachments for illness-based claim",
  "confidence_level": 0.7,
  "summary_of_findings": "The claim is for trip cancellation due to the insured's illness (cancer). The policy covers cancellation for an unforeseen Sickness of the Insured. However, the policy explicitly requires medical documentation, including a Medical Certificate form, to process such a claim. No medical evidence was provided among the attachments. Therefore, the claim is declined for failure to provide required proof of loss. There is a discrepancy between the trip cost in the policy data ($283.38) and the claimed amount ($664.12), but this is secondary to the lack of evidence.",
  "case_description": "The insured filed a trip cancellation claim for $664.12 due to a recent cancer diagnosis and subsequent treatment, which made travel unfeasible. The cancellation occurred prior to the scheduled departure. The policy provides coverage for such events, but is contingent on receiving medical documentation, which was not supplied with the claim."
}
